In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
!pip install transformers

In [21]:
!pip install emoji

In [22]:
import tensorflow as tf
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, BertTokenizer
from tqdm import tqdm, trange
import pandas as pd
import io
import os
import re
import emoji
import random

import nltk
nltk.download('stopwords')
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, TweetTokenizer
from nltk.corpus import wordnet, stopwords

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
from statistics import mode


# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'Tesla V100-SXM2-16GB'

In [23]:
def preprocess(df):
    
    #removes URL
    pattern = r'https.?://[^\s]+[\s]?'
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes usernames/mentions
    pattern = r'@[^\s]+'
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes emoji and smiley
    pattern = re.compile("["
                         u"\U0001F600-\U0001F64F"
                         u"\U0001F300-\U0001F5FF"
                         u"\U0001F680-\U0001F6FF"
                         u"\U0001F1E0-\U0001F1FF"
                         u"\U00002500-\U00002BEF"
                         u"\U00002702-\U000027B0"
                         u"\U00002702-\U000027B0"
                         u"\U000024C2-\U0001F251"
                         u"\U0001f926-\U0001f937"
                         u"\U00010000-\U0010ffff"
                         u"\u2640-\u2642"
                         u"\u2600-\u2B55"
                         u"\u200d"
                         u"\u23cf"
                         u"\u23e9"
                         u"\u231a"
                         u"\ufe0f"
                         u"\u3030"
                         "]+", flags=re.UNICODE)
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes numbers
    pattern = r'\d+'
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
    
    #removes punctuation
    pattern = r"[^\w\s]"
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl=" ", regex=True)

    #removes stop words
    stop_words = stopwords.words("english")    
    remove_stop_words = lambda row: " ".join([token for token in row.split(" ")
                                              if token not in stop_words])
    df["tweet"] = df["tweet"].apply(remove_stop_words)
    
    #removes extra spaces
    pattern = r"[\s]+"
    df["tweet"] = df["tweet"].str.replace(pat=pattern, repl=" ", regex=True)
    
    return(df)

In [24]:
def train_validate_test_split(df,seed, train_percent=.8, validate_percent=.125):
  train, test = train_test_split(df, train_size=train_percent, stratify=df['label'], random_state=seed)
  train, validate = train_test_split(train, test_size=validate_percent, stratify=train['label'], random_state=seed)
  return train, validate, test

def sample_data(df,sample,seed):
    X_train, _, y_train, _ = train_test_split( df['tweet'], df['label'], train_size=sample, random_state=seed, stratify=df['label'])
    return pd.concat([X_train,y_train], axis = 1 )

def tokenize_data(df):
    sentences = ["[CLS] " + query + " [SEP]" for query in df['tweet']]
    # Tokenize with multilingual BERT tokenizer
    tokenizer = BertTokenizer.from_pretrained('asafaya/bert-base-arabic', do_lower_case=True)
    #tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    
    MAX_LEN = 128

    # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    # Create attention masks
    attention_masks = []
    # Create a mask of 1s for each token followed by 0s for padding
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return input_ids, attention_masks

def Data_Loader(inputs_ids, attention_masks, df,batch_size=16): 
    data = TensorDataset(torch.LongTensor(inputs_ids), torch.LongTensor(attention_masks), torch.LongTensor(df['label'].values))
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
    return dataloader

In [25]:
def model_train(model, train_dataloader, validation_dataloader):
    # BERT training loop
    epochs = 3
    for _ in trange(epochs, desc="Epoch"):  
        # Set our model to training mode
        model.train()
        # Tracking variables
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        # Train the data for one epoch
        for step, batch in enumerate(train_dataloader):
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs["loss"]
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            # Update tracking variables
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1
        print("Train loss: {}".format(tr_loss/nb_tr_steps))

        ## VALIDATION

        # Put model in evaluation mode
        model.eval()
        # Tracking variables 
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0
        # Evaluate data for one epoch
        for batch in validation_dataloader:
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            # Telling the model not to compute or store gradients, saving memory and speeding up validation
            with torch.no_grad():
              # Forward pass, calculate logit predictions
                logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)[0]
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            tmp_eval_accuracy = flat_accuracy(logits, label_ids)    
            eval_accuracy += tmp_eval_accuracy
            nb_eval_steps += 1
        validation_accuracy = (eval_accuracy/nb_eval_steps)
        print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    return validation_accuracy

In [26]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def model_test(model,prediction_dataloader):
    model.eval()
    # Tracking variables 
    predictions , true_labels = [], []
# Predict
    for batch in prediction_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up prediction
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions+=list(np.argmax(logits, axis=1).flatten())
        true_labels+=list(label_ids.flatten())
    test_f1_score = f1_score(true_labels, predictions, average= 'macro')
    print("Macro F1 Score:",test_f1_score)
    test_f1_wscore = f1_score(true_labels, predictions, average= 'weighted')
    print("Weighted F1 Score:",test_f1_wscore)
    test_accuracy_score = accuracy_score(true_labels, predictions)
    print("Accuracy score:", test_accuracy_score, "\n")
    print("="*100)
    return test_f1_score, test_f1_wscore

In [27]:
def model_initialise():
  # Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
  model = BertForSequenceClassification.from_pretrained("asafaya/bert-base-arabic", num_labels=2).cuda()
  optimizer = optim.AdamW(params = model.parameters(), lr=2e-5)
  return model, optimizer

In [28]:
print_stmts= []
languages = ['Arabic']
directory = 'drive/My Drive/CS695'
for lang in languages:
  df = pd.read_csv(os.path.join(directory, lang+'.csv'))
  df = preprocess(df)
  sample_sizes = [16, 32, 64, 128, 256]
for sample in sample_sizes: 
  seeds = [2018,2019, 2020, 2021, 2022]
  weighted = []
  macro = []
  for seed in seeds:
    np.random.seed(seed)
    train_df, validation_df, test_df = train_validate_test_split(df, seed)
    train_len = len(train_df)
    if sample==256 and seed==2022:
        sample_sizes.append(train_len)
    if sample == train_len and seed == 2022:
        sample_sizes.remove(train_len)
    model, optimizer = model_initialise()
    if(sample != train_len):  
      train_df_sample = sample_data(train_df,sample,seed)
      train_input_ids, train_attention_masks = tokenize_data(train_df_sample)
      train_dataloader = Data_Loader(train_input_ids, train_attention_masks, train_df_sample)
    else:
      train_input_ids, train_attention_masks = tokenize_data(train_df)
      train_dataloader = Data_Loader(train_input_ids, train_attention_masks, train_df)

    validation_input_ids, validation_attention_masks = tokenize_data(validation_df)
    validation_dataloader = Data_Loader(validation_input_ids, validation_attention_masks, validation_df)
    print("\nModel Summary:")
    print('Language:', lang)
    print('Sample Size:', sample)
    print('Seed value:', seed)
    validation_accuracy = model_train(model, train_dataloader, validation_dataloader)
    test_input_ids, test_attention_masks = tokenize_data(test_df)
    test_dataloader = Data_Loader(test_input_ids, test_attention_masks, test_df)
    m, w = model_test(model, test_dataloader)
    weighted.append(w)
    macro.append(m)
  print("The Average  Weighted F1-Score of the Language ", lang, "is:",sum(weighted)/ len(weighted))
  print("The Average  Macro F1-Score of the Language ", lang, "is:",sum(macro)/ len(macro))
  print("="*200)
  print_stmts.append("For Sample Size "+str(sample)+" Average Weighted F1-Score "+str(sum(weighted)/len(weighted))+" and Average Macro F1-Score "+str(sum(macro)/len(macro))+" of "+ str(lang))
for i in print_stmts:
  print(i,"\n")
print("="*100+str(lang)+"="*100)

Some weights of the model checkpoint at asafaya/bert-base-arabic were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at asafaya/bert-base-ar


Model Summary:
Language: Arabic
Sample Size: 16
Seed value: 2018
Train loss: 0.761111319065094


Epoch:  33%|███▎      | 1/3 [00:01<00:02,  1.48s/it]

Validation Accuracy: 0.7381756756756757
Train loss: 0.6452572345733643


Epoch:  67%|██████▋   | 2/3 [00:02<00:01,  1.48s/it]

...

Epoch:  67%|██████▋   | 2/3 [01:12<00:36, 36.40s/it]

Validation Accuracy: 0.8811936936936936
Train loss: 0.09850293807740375


Epoch: 100%|██████████| 3/3 [01:49<00:00, 36.41s/it]

Validation Accuracy: 0.893581081081081


Macro F1 Score: 0.8471050486930947
Weighted F1 Score: 0.8988968959624088
Accuracy score: 0.8998272884283247 

The Average  Weighted F1-Score of the Language  Arabic is: 0.9068040773148063
The Average  Macro F1-Score of the Language  Arabic is: 0.8605277976324324
For Sample Size 16 Average Weighted F1-Score 0.6951569873964752 and Average Macro F1-Score 0.44084983099951713 of Arabic 

For Sample Size 32 Average Weighted F1-Score 0.6951569873964752 and Average Macro F1-Score 0.44084983099951713 of Arabic 

For Sample Size 64 Average Weighted F1-Score 0.7058779388409632 and Average Macro F1-Score 0.464008068647115 of Arabic 

For Sample Size 128 Average Weighted F1-Score 0.7563754582809123 and Average Macro F1-Score 0.5672015800105791 of Arabic 

For Sample Size 256 Average Weighted F1-Score 0.8567085672292182 and Average Macro F1-Score 0.7796109893936248 of Arabic 

For Sample Size 4051 Average Weighted F1-Score 0.9068040773148063 and Average Macro F1-Score 0.8605277976324324 of Arabic 

